In [1]:
import warnings

warnings.filterwarnings("ignore")

import os
import re
import sys
from collections import Counter
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from rich import print
from tqdm import tqdm

In [2]:
def read_file_to_str_li(fp, print_exp=True):
    with open(fp, "r") as f:
        lines = f.read().split("\n")
    if print_exp:
        print(f"Read from {fp}:")
        print(f"First line: {lines[0]} | Last line: {lines[-1]}")
        print("-" * 6)

    return lines


# define the function blocks
def convert_to_int(input_str):
    if input_str == "" or input_str == " ":
        return None
    return int(input_str)


def convert_to_str(input_str):
    return str(input_str)


# map the inputs to the function blocks
converts = {
    "i": convert_to_int,
    "s": convert_to_str,
}


def convert_str_li_to_other_li(
    str_li, pattern="i", per_letter=False, sep=" ", start_row=0, end_row=None
):
    """ Convert a list of string to a list of other types
    
    pattern: a list of types for one item. 
        'i' for int, 's' for string
        'si' means: convert the 1st item to string, the rest to integer
        If separated items are more than pattern items,
        use the last one from the parttern.
    if per_letter=True, ignore sep and separate item per letter
    """
    target_str_li = str_li[start_row:end_row]
    # find max item num
    max_item_num = 1
    if per_letter:
        max_item_num = max([len(s) for s in target_str_li])
    else:
        max_item_num = max([len(s.split(sep)) for s in target_str_li])

    # extend the pattern to the max itme num
    pattern = (
        pattern + f"{pattern[-1]}" * (max_item_num - len(pattern))
        if max_item_num > len(pattern)
        else pattern
    )

    # convert
    if per_letter:
        return [
            [converts[pattern[idx]](item) for idx, item in enumerate(s)]
            for s in target_str_li
        ]
    else:
        if sep == " ":
            return [
                [converts[pattern[idx]](item) for idx, item in enumerate(s.split())]
                for s in target_str_li
            ]
        else:
            return [
                [converts[pattern[idx]](item) for idx, item in enumerate(s.split(sep))]
                for s in target_str_li
            ]

In [3]:
fp = "input.txt"
lines = read_file_to_str_li(fp)

print("Convert to:")

head = convert_str_li_to_other_li(
    lines, pattern="s", per_letter=True, sep=",", start_row=0, end_row=1
)

print(f"Head:\n{head}")
# print(f"First line: {head[0]}")
# print(f"Last line: {head[-1]}")

# data = convert_str_li_to_other_li(
#     lines, pattern="i", per_letter=True, sep=" ", start_row=None, end_row=None
# )
data = convert_str_li_to_other_li(
    lines, pattern="s", per_letter=False, sep=" -> ", start_row=2, end_row=None
)
# data = convert_str_li_to_other_li(
#     lines, pattern="i", per_letter=False, sep=",", start_row=None, end_row=None
# )
# data = convert_str_li_to_other_li(
#     lines, pattern="s", per_letter=False, sep=" -> ", start_row=None, end_row=None
# )
# data = convert_str_li_to_other_li(
#     lines, pattern="i", per_letter=False, sep=" ", start_row=2, end_row=None
# )
# data = convert_str_li_to_other_li(
#     lines, pattern="si", per_letter=False, sep=" ", start_row=0, end_row=None
# )
# data = convert_str_li_to_other_li(
#     lines, pattern="i", per_letter=True, sep=" ", start_row=0, end_row=None
# )

print(f"First line: {data[0]}")
print(f"Last line: {data[-1]}")
print("-" * 6)

Read from input.txt:

First line: VFHKKOKKCPBONFHNPHPN | Last line: KB -> V

------

Convert to:

Head:
[['V', 'F', 'H', 'K', 'K', 'O', 'K', 'K', 'C', 'P', 'B', 'O', 'N', 'F', 'H', 'N', 'P', 'H', 
'P', 'N']]

First line: ['VS', 'B']

Last line: ['KB', 'V']

------

In [4]:
template = {k: v for k, v in data}

In [5]:
# Slow solution

elements = "".join(head[0])


def check_add_ele(elements, template):
    add_ele = {}
    for i in range(len(elements) - 1):
        key = elements[i : i + 2]
        if key in template:
            add_ele[i] = template[key]
    return add_ele


def add_new_ele(elements, add_ele):
    new_ele = ""
    for idx, e in enumerate(elements):
        new_ele += e
        if idx in add_ele:
            new_ele += add_ele[idx]
    return new_ele


def loop_step(step, elements, template):
    for _ in tqdm(range(step)):
        elements = add_new_ele(elements, check_add_ele(elements, template))
    return elements


new_ele = loop_step(10, elements, template)

count_li = Counter(new_ele).most_common()
print(f"Answer to Q1: {count_li[0][1]-count_li[-1][1]}")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 833.53it/s]


Answer to Q1: 2988

In [6]:
# Fast solution

elements = "".join(head[0])
ele_dict = {}

for i in range(len(elements) - 1):
    key = elements[i : i + 2]
    if key in ele_dict:
        ele_dict[key] += 1
    else:
        ele_dict[key] = 1


def add_new_ele(ele_dict, template):
    new_ele_dict = ele_dict.copy()
    for k, v in ele_dict.items():
        if k in template:
            new_items = [k[0] + template[k], template[k] + k[1]]
            for new_item in new_items:
                if new_item in new_ele_dict:
                    new_ele_dict[new_item] += v
                else:
                    new_ele_dict[new_item] = v
            new_ele_dict[k] -= v
    return new_ele_dict


def loop_step(step, ele_dict, template):
    for _ in tqdm(range(step)):
        ele_dict = add_new_ele(ele_dict, template)
    return ele_dict


ele_dict = loop_step(40, ele_dict, template)
# ele_dict

count_e = {}
count_e["V"] = 1
count_e["N"] = 1

for k, v in ele_dict.items():
    for it in k:
        if it in count_e:
            count_e[it] += v
        else:
            count_e[it] = v

# count_e

li = np.array(list(count_e.values()))

print(f"Answer to Q2: {int((li.max()-li.min())/2)}")

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 13327.94it/s]


Answer to Q2: 3572761917024